In [59]:
import requests
import os

from mutagen.id3 import ID3, APIC
import mutagen


import json

base_url = "https://api-v2.soundcloud.com"
resolve_url = f"{base_url}/resolve"
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0"
default_headers = {
    "User-Agent" : user_agent
}


def resolve(client_id, track_url):
    params = {
        "client_id" : client_id,
        "url" : track_url
    }

    response = requests.get(resolve_url, params = params, headers = default_headers)
    return response.json()

def get_streaming_url(client_id, res):
    has_prog = False
    for tr in res['media']['transcodings']:
        if tr['format']['protocol'] == 'progressive':
            prog_url = tr['url']
            has_prog = True
    if has_prog == False:
        print("no progressive streaming found -- download likely broken. will try anyways")
        hls_url = tr['url']   


    if has_prog:
        result = requests.get(
            prog_url,
            params = {
                "client_id" : client_id
            },
            headers = default_headers
        )
        stream_url = result.json()['url']
    else:
        result = requests.get(
            hls_url,
            params = {
                "client_id" : client_id
            },
            headers = default_headers
        )
        stream_url = result.json()['url']
    return stream_url

def stream_download(stream_url, title):
    stream = requests.get(stream_url, stream = True)

    os.makedirs('dls', exist_ok=True)
    filepath = os.path.join('dls', f"{title}.mp3")

    with open(filepath, 'wb') as output:
        output.write(stream.content)
        print(f"{title}.mp3, size: {round(os.stat(filepath).st_size / (1024*1024), 2)} mb.")
    
    return filepath


def add_metadata(res, filepath):
    # get cover img, save to 'imgpath'
    cover_img = requests.get(res['artwork_url']).content
    os.makedirs('dls', exist_ok=True)
    imgpath = os.path.join('dls', f"coverart.jpg")
    with open(imgpath, 'wb') as img:
        img.write(cover_img)
    
    # add title and artist
    audio_ez = mutagen.File(filepath, easy = True)

    if audio_ez.tags is None:
        audio_ez.add_tags()
    audio_ez['title'] = res['title']
    audio_ez['artist'] = res['user']['username']
    audio_ez.save()

    # add cover art - can't use easyID3

    audio = mutagen.File(filepath)
    with open(imgpath, 'rb') as coverart:
        audio['APIC'] = APIC(
            encoding = 3,
            mime = 'image/jpeg',
            type = 3,
            desc = u'Cover',
            data = coverart.read()
        )
    audio.save()

    os.remove(imgpath)




In [60]:
client_id = "tPycpzX7dXV3LN9SC9RpDUI9s4lKl9cc"  # Replace with your actual client_id
resource_url = "https://soundcloud.com/jameskrivchenia/15a"  # Example SoundCloud URL

res = resolve(client_id, resource_url)

a = get_streaming_url(client_id, res)

filepath = stream_download(a, f"{res['user']['username']} - {res['title']}")
add_metadata(res, filepath)

James Krivchenia - 1000000000s - Effective Dream.mp3, size: 1.21 mb.


In [55]:
print(a)

https://cf-media.sndcdn.com/VtBjm3ZEQdtX.128.mp3?Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiKjovL2NmLW1lZGlhLnNuZGNkbi5jb20vVnRCam0zWkVRZHRYLjEyOC5tcDMqIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoxNzA5MjAyNDc3fX19XX0_&Signature=TIchkb6m6FRwKiYwf6XXGVT5NOWWLH3JEQ~Ue5ejapW8gm0rwZTgM7yHFn0o3vq61oFiCEozUQb59z2WqXnSJ96YdjAo61n072Leo5yrkfTojrCzR-h9hzIYGFtOy1WTuVsvdYxdQo8oecVIJQOfmkNhkzGTN69vW8v6eK07ZJ8zlWni2sQr~cPlax3YMrf7l1gHpbTT-RuppkbgC8UEcfjAu8w~auw~uutdJZ0yZRHixz11bSqrqTeRe3MstUY8gPJ1BKMQ9tv3-lnzwIcxYSzNV-mzRtRArW8Ax9DRZIRi-YBr4-YFiAp3SEzHVtuzIV8kOyJQhpY9Mx3DbJpZxA__&Key-Pair-Id=APKAI6TU7MMXM5DG6EPQ


In [20]:
{
  "artwork_url": "https://i1.sndcdn.com/artworks-000155844329-24x5gy-large.jpg",
  "caption": null,
  "commentable": true,
  "comment_count": 0,
  "created_at": "2016-03-28T16:20:45Z",
  "description": "CDs available from Reading Group -- http://readinggroup.co/",
  "downloadable": false,
  "download_count": 0,
  "duration": 79106,
  "full_duration": 79106,
  "embeddable_by": "all",
  "genre": "Electronic",
  "has_downloads_left": false,
  "id": 255594431,
  "kind": "track",
  "label_name": null,
  "last_modified": "2018-06-13T17:29:58Z",
  "license": "all-rights-reserved",
  "likes_count": 19,
  "permalink": "15a",
  "permalink_url": "https://soundcloud.com/jameskrivchenia/15a",
  "playback_count": 639,
  "public": true,
  "publisher_metadata": {
    "id": 255594431,
    "urn": "soundcloud:tracks:255594431"
  },
  "purchase_title": null,
  "purchase_url": null,
  "release_date": null,
  "reposts_count": 3,
  "secret_token": null,
  "sharing": "public",
  "state": "finished",
  "streamable": true,
  "tag_list": "",
  "title": "1000000000s - Effective Dream",
  "track_format": "single-track",
  "uri": "https://api.soundcloud.com/tracks/255594431",
  "urn": "soundcloud:tracks:255594431",
  "user_id": 980154,
  "visuals": null,
  "waveform_url": "https://wave.sndcdn.com/VtBjm3ZEQdtX_m.json",
  "display_date": "2017-01-15T19:52:31Z",
  "media": {
    "transcodings": [
      {
        "url": "https://api-v2.soundcloud.com/media/soundcloud:tracks:255594431/e1ea5b3d-850e-49a1-a95b-dbdcd8ebea4f/stream/hls",
        "preset": "mp3_0_0",
        "duration": 79106,
        "snipped": false,
        "format": {
          "protocol": "hls",
          "mime_type": "audio/mpeg"
        },
        "quality": "sq"
      },
      {
        "url": "https://api-v2.soundcloud.com/media/soundcloud:tracks:255594431/e1ea5b3d-850e-49a1-a95b-dbdcd8ebea4f/stream/progressive",
        "preset": "mp3_0_0",
        "duration": 79106,
        "snipped": false,
        "format": {
          "protocol": "progressive",
          "mime_type": "audio/mpeg"
        },
        "quality": "sq"
      },
      {
        "url": "https://api-v2.soundcloud.com/media/soundcloud:tracks:255594431/2e22274b-1582-450d-8bde-2e90cf4ce860/stream/hls",
        "preset": "opus_0_0",
        "duration": 79106,
        "snipped": false,
        "format": {
          "protocol": "hls",
          "mime_type": "audio/ogg; codecs=\"opus\""
        },
        "quality": "sq"
      }
    ]
  },
  "station_urn": "soundcloud:system-playlists:track-stations:255594431",
  "station_permalink": "track-stations:255594431",
  "track_authorization": "[REDACTED]",
  "monetization_model": "BLACKBOX",
  "policy": "MONETIZE",
  "user": {
    "avatar_url": "https://i1.sndcdn.com/avatars-mgygCF2636EWKkcq-BUpcVw-large.jpg",
    "city": "New Mexico",
    "comments_count": 0,
    "country_code": "US",
    "created_at": "2010-05-08T19:48:24Z",
    "creator_subscriptions": [
      {


b'{"artwork_url":"https://i1.sndcdn.com/artworks-000155844329-24x5gy-large.jpg","caption":null,"commentable":true,"comment_count":0,"created_at":"2016-03-28T16:20:45Z","description":"CDs available from Reading Group -- http://readinggroup.co/","downloadable":false,"download_count":0,"duration":79106,"full_duration":79106,"embeddable_by":"all","genre":"Electronic","has_downloads_left":false,"id":255594431,"kind":"track","label_name":null,"last_modified":"2018-06-13T17:29:58Z","license":"all-rights-reserved","likes_count":19,"permalink":"15a","permalink_url":"https://soundcloud.com/jameskrivchenia/15a","playback_count":638,"public":true,"publisher_metadata":{"id":255594431,"urn":"soundcloud:tracks:255594431"},"purchase_title":null,"purchase_url":null,"release_date":null,"reposts_count":3,"secret_token":null,"sharing":"public","state":"finished","streamable":true,"tag_list":"","title":"1000000000s - Effective Dream","track_format":"single-track","uri":"https://api.soundcloud.com/tracks/25

In [ ]:
from soundcloud import SoundCloud
import requests

import os

from mutagen.id3 import ID3, APIC
import mutagen

class Downloader:
    BASE_URL = "https://api-v2.soundcloud.com"
    RESOLVE_URL = f"{base_url}/resolve"
    USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0"
    def __init__(self, client_id):
        self.client = SoundCloud(client_id)


    def get_resolved(self, track_url):
        params = {
            "client_id" : client_id,
            "url" : track_url
        }

        response = requests.get(resolve_url, params = params, headers = default_headers)
        return response.json()
        
    def get_streaming_url(self, track):
        has_prog = False

        for tr in track.media.transcodings:
            if tr.format.protocol == 'progressive':
                prog_url = tr.url
                has_prog = True
                break
        if not has_prog:
            print("no progressive streaming found -- download likely broken. will try anyways")
            hls_url = tr.url

        headers = self.client.get_default_headers()
        if has_prog:
            json = requests.get(prog_url, params={"client_id": self.client.client_id}, headers=headers)
            stream_url = json.json()["url"]
        else:
            json = requests.get(hls_url, params={"client_id": self.client.client_id}, headers=headers)
            stream_url = json.json()["url"]

        return stream_url

    def stream_download(self, stream_url, dest):
        stream = requests.get(stream_url)

        with open(dest, 'wb') as output:
            output.write(stream.content)
            print(f"{dest}, size: {round(os.stat(dest).st_size / (1024*1024), 2)} mb.")

        return stream_url

    def add_metadata(self, track, dest):
        # get cover img, save to 'imgpath'
        cover_img = requests.get(track.artwork_url).content
        imgpath = "coverart.jpg"
        with open(imgpath, 'wb') as img:
            img.write(cover_img)
        
        # add title and artist
        audio_ez = mutagen.File(dest, easy = True)

        if audio_ez.tags is None:
            audio_ez.add_tags()
        audio_ez['title'] = track.title
        audio_ez['artist'] = track.user.username
        audio_ez.save()

        # add cover art - can't use easyID3

        audio = mutagen.File(dest)
        with open(imgpath, 'rb') as coverart:
            audio['APIC'] = APIC(
                encoding = 3,
                mime = 'image/jpeg',
                type = 3,
                desc = u'Cover',
                data = coverart.read()
            )
        audio.save()

        os.remove(imgpath)
        return audio_ez['title'], audio_ez['artist']

    def playlist_to_tracks(self, track):
        track_urls = []
        print("gathering tracks from set...")
        for t in track.tracks:
            try:
                track_urls.append(t.permalink_url)
            except AttributeError:
                track_urls.append(self.client.get_track(t.id).permalink_url)
        print(f"{len(track_urls)} tracks found.")
        return track_urls


In [64]:
res = resolve(client_id, "https://soundcloud.com/jameskrivchenia/sets/youre-useless-i-love-you")

In [72]:
for t in res['tracks']:
    print(t['id'])

255594416
255594450
255594455
255594457
255594459
255594465
255594462
255594469
255594471
255594412
255594419
255594421
255594428
255594436
255594431
255594439
255594445
255594448


In [82]:
track_urls = []
for t in res['tracks']:
    try:
        track_urls.append(t['permalink_url'])
    except KeyError:
        track_urls.append(
            requests.get(
                f"https://api-v2.soundcloud.com/tracks/{t['id']}", 
                params = {"client_id": client_id}, 
                headers = default_headers
                ).json()['permalink_url']
            )



https://soundcloud.com/jameskrivchenia/1a-1
https://soundcloud.com/jameskrivchenia/2a-1
https://soundcloud.com/jameskrivchenia/3a-1
https://soundcloud.com/jameskrivchenia/4a-1
https://soundcloud.com/jameskrivchenia/5a-1


In [84]:
print(result['permalink_url'])

https://soundcloud.com/jameskrivchenia/18a
